## 將Bitstream File 燒錄至板子上

In [3]:
from pynq import Overlay
Lab_Design = Overlay("./HW6_v2.bit")

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


## 從硬體檔案中讀取這個系統中的 AXI ADDRESS

In [4]:
Lab_Design.ip_dict

{'convolution_0': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'convolution_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'SEG_convolution_0_S00_AXI_reg',
  'phys_addr': 1136656384,
  'state': None,
  'type': 'xilinx.com:user:convolution:1.0'}}

## 取出 ADDER 這個 AXI IP 的 Base Address 以及 Address Range
### 可以檢查這個地址是否和Vivado中 `Address Editor` 內 ADDER 的 Base Address 相同

In [7]:
convolution_CON = Lab_Design.ip_dict['convolution_0']['phys_addr']
convolution_CON_range = Lab_Design.ip_dict['convolution_0']['addr_range']
print("The Base Address of convolution is : " + format(convolution_CON, '02x'))
print("The Address Range of convolution is : " + format(convolution_CON_range, '02x'))

The Base Address of convolution is : 43c00000
The Address Range of convolution is : 10000


## 使用Pynq 內建的 MMIO 以達成 PS 和 AXI IP 之間的溝通
### MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

In [8]:
from pynq import MMIO
convolution = MMIO(convolution_CON, convolution_CON_range)

## 輸入加數及被加數，並做輸入資料的前處理

In [12]:
x0 = 8
x1 = 7
x2 = 6
x3 = 5
w0 = 8
w1 = 7
w2 = 6
w3 = 5
b = 3
input_data = x0 + (x1 << 4) + (x2 << 8) + (x3 << 12) + (w0 << 16) + (w1 << 20) + (w2 << 24) + (w3 << 28)

## 將輸入資料寫進ADDER offset 為 0 的位置
## 並從 offset 為 4 的位置讀出運算結果

In [13]:
convolution.write(0x0, input_data)
convolution.write(0x4, b)
output = convolution.read(0x8)
print("The answer is ", output)

The answer is  177
